## <span style="color:purple">Using SyntaxIgnoreTagger and SyntaxIgnoreCutter in pre-processing for syntactic analysis</span>

Some parts of a text may be difficult to analyse syntactically. For instance, enumerations of sports results, and content in parentheses (references and short remarks) may exhibit only very little linguistic structure suitable for analysis. So, you may want to skip the syntactic analysis on such parts of the text.

In order to detect parts of text that should be ignored by the syntactic analysis, EstNLTK has a special tagger called SyntaxIgnoreTagger.

After ignorable parts have been detected, you can use SyntaxIgnoreCutter to create a new Text object without these parts, and apply syntactic analysis.
And the resulting syntactic analysis layer can also be carried over to the original text, if needed.

## SyntaxIgnoreTagger 

In [1]:
from estnltk.taggers.standard.syntax.preprocessing.syntax_ignore_tagger import SyntaxIgnoreTagger
syntax_ignore_tagger = SyntaxIgnoreTagger()
syntax_ignore_tagger

SyntaxIgnoreTagger(input_layers=('words', 'sentences'), output_layer=syntax_ignore, output_attributes=('type',), allow_loose_match=True, ignore_parenthesized_num=True, ignore_parenthesized_num_greedy=True, ignore_parenthesized_ref=True, ignore_parenthesized_title_words=True, ignore_parenthesized_short_char_sequences=True, ignore_consecutive_parenthesized_sentences=True, ignore_consecutive_enum_ucase_num_sentences=True, ignore_sentences_consisting_of_numbers=True, ignore_sentences_starting_with_time=True, ignore_sentences_with_comma_separated_num_name_lists=True, ignore_brackets=True)

SyntaxIgnoreTagger requires that 'words' and 'sentences' have been annotated in text. It uses these layers to detect snippets of text (not necessarily full sentences) which should be ignored during the syntactic analysis. Detection patterns are partly based on the patterns used in pre-processing modules of EstSyntax, available [here](https://github.com/kristiinavaik/ettenten-eeltootlus) and [here](https://github.com/EstSyntax/preprocessing-module).

Similar detection patterns have been grouped together. Upon the initialization of the tagger, flags can be used to switch these groups off (by default, all flags have been switched on). In following, we will give a minute introduction on the flags, and types of ignorable text snippets the corresponding patterns aim to extract.

### Patterns for detecting ignore content inside sentences

#### Flag `ignore_brackets`

Content inside square brackets will be ignored.

In [2]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Nurksulgudes tuuakse materjali viitekirje järjekorranumber kirjanduse loetelus ja leheküljed , nt [9: 5] või [9 lk 5], aga internetimaterjalil lihtsalt viitekirje, nt [7]')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_brackets=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['[', '9', ':', '5', ']'], [{'type': 'brackets_ref'}]),
EnvelopingSpan(['[', '9', 'lk', '5', ']'], [{'type': 'brackets_ref'}]),
EnvelopingSpan(['[', '7', ']'], [{'type': 'brackets_ref'}])])

#### Flag `ignore_parenthesized_ref`

Parenthesized content which looks like a reference (e.g. contains titlecased words, and date information) will be ignored.

In [3]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Tutvustamisele tulevad Jan Kausi romaan “Koju” (Tuum 2012) ning Ülo Pikkovi romaan “Vana prints” (Varrak 2012). Temaatikaga seondub veel teinegi äsja Postimehes ilmunud jutt (Priit Pullerits «Džiibi kaitseks», PM 30.07.2010).')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_ref=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', 'Tuum', '2012', ')'], [{'type': 'parentheses_ref_year'}]),
EnvelopingSpan(['(', 'Varrak', '2012', ')'], [{'type': 'parentheses_ref_year'}]),
EnvelopingSpan(['(', 'Priit', 'Pullerits', '«', 'Džiibi', 'kaitseks', '»', ',', 'PM', '30.07.2010', ')'], [{'type': 'parentheses_ref_year'}])])

#### Flag `ignore_parenthesized_short_char_sequences`

Parenthesized short sequences of tokens (up to 4 tokens), each of which also has as short length (up to 4 characters), will be ignored.

In [4]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Eesti judokate võistlus jäi laupäeval lühikeseks , nii Joel Rothberg ( -66 kg ) kui ka Renee Villemson ( -73 kg ) võidurõõmu maitsta ei saanud .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_short_char_sequences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', '-66', 'kg', ')'], [{'type': 'parentheses_1to3'}]),
EnvelopingSpan(['(', '-73', 'kg', ')'], [{'type': 'parentheses_1to3'}])])

#### Flag `ignore_parenthesized_title_words`

Parenthesized 1-2 titlecase words (which may be comma-separated) will be ignored.

In [5]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Neidude 5 km klassikat võitis Lina Andersson ( Rootsi ) Pirjo Mannineni ( Soome ) ja Karin Holmbergi ( Rootsi ) ees .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_title_words=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', 'Rootsi', ')'], [{'type': 'parentheses_title_words'}]),
EnvelopingSpan(['(', 'Soome', ')'], [{'type': 'parentheses_title_words'}]),
EnvelopingSpan(['(', 'Rootsi', ')'], [{'type': 'parentheses_title_words'}])])

#### Flag `ignore_parenthesized_num`

Parenthesized numerics (such as dates, date ranges, number sequences) will be ignored.

In [6]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Klubi sai kuus korda Inglismaa meistriks (1976, 1977, 1979, 1980, 1982, 1983). Tallinna ( 21.-22. mai ) , Haapsalu ( 2.-3. juuli ) ja Liivimaa ( 30.-31. juuli ) rallidel on vähemalt see probleem lahendatud .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_num=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', '1976', ',', '1977', ',', '1979', ',', '1980', ',', '1982', ',', '1983', ')'], [{'type': 'parentheses_num'}]),
EnvelopingSpan(['(', '21.', '-', '22.', 'mai', ')'], [{'type': 'parentheses_num_range'}]),
EnvelopingSpan(['(', '2.', '-', '3.', 'juuli', ')'], [{'type': 'parentheses_num_range'}]),
EnvelopingSpan(['(', '30.', '-', '31.', 'juuli', ')'], [{'type': 'parentheses_num_range'}])])

#### Flag `ignore_parenthesized_num_greedy`

Applies greedy parenthesized numeric content detection patterns: if there is at least one number inside parentheses, but there cannot be found at least 3 consecutive lowercase words, then the whole content inside parentheses will be marked as to be ignored.

In [7]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Näited: A (300 000 000 m/sek), B ( naised 5 km , mehed 7,5 km ) ning C (meie puhul 165/80) .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_num_greedy=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', '300 000 000', 'm/sek', ')'], [{'type': 'parentheses_num_start_uncategorized'}]),
EnvelopingSpan(['(', 'naised', '5', 'km', ',', 'mehed', '7,5', 'km', ')'], [{'type': 'parentheses_num_mid_uncategorized'}]),
EnvelopingSpan(['(', 'meie', 'puhul', '165', '/', '80', ')'], [{'type': 'parentheses_num_end_uncategorized'}])])

Note that this pattern group also covers content detected by some other `ignore_parenthesized_*` patterns. So, if you want to turn other patterns off, you may also want to turn off this pattern.

#### Flag `allow_loose_match`

If `True` (default setting), then an ignore text snippet may consume words without matching exactly with their boundaries (e.g. ignore snippet's start does not have to match word's start). If `False`, then an ignore text snippet must match exactly with word boundaries: it must start where a word starts, and end where a word ends.

### Patterns for ignoring full sentences

#### Flag  `ignore_sentences_starting_with_time`

Sentences starting with a date range (e.g. a time schedule of a seminar, or a TV program) will be ignored.

In [8]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''
12.05 - 12.35 "Õnne 13" (1. osa)
12.35 - 13.05 "Õnne 13" (1. osa kordus)
13.05 - 13.35 "Õnne 13" (2. osa)
''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_sentences_starting_with_time=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['12.05', '-', '12.35', '"', 'Õnne', '13', '"', '(', '1.', 'osa', ')', '12.35', '-', '13.05', '"', 'Õnne', '13', '"', '(', '1.', 'osa', 'kordus', ')', '13.05', '-', '13.35', '"', 'Õnne', '13', '"', '(', '2.', 'osa', ')'], [{'type': 'sentence_starts_with_time'}])])

#### Flag `ignore_sentences_with_comma_separated_num_name_lists`

Sentences containing comma separated list of titlecase words / numbers ( like sport results, player/country listings, game scores etc.) will be ignored.

In [9]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''
Veerandfinaalid .
Lindsay Davenport ( 6 ) , USA-Jana Novotna ( 3 ) , Tšehhi 6 : 2 , 4 : 6 , 7 : 6 , Martina Hingis ( 1 ) , Šveits-Arantxa Sanchez Vicario ( 10 ) , Hispaania 6 : 3 , 6 : 2 .
Paarismängu veerandfinaalid .
Gigi Fernandez , USA/Nataša Zvereva , Valgevene- Alexandra Fusai/Nathalie Tauziat , Prantsusmaa 4 : 6 , 6 : 2 , 6 : 2 , Nicole Arendt , USA/Manon Bollegraf , Holland-Ruxandra Dragomir , Rumeenia/Iva Majoli , Horvaatia 6 : 3 , 3 : 6 , 6 : 4 .
''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_sentences_with_comma_separated_num_name_lists=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['Lindsay', 'Davenport', '(', '6', ')', ',', 'USA-Jana', 'Novotna', '(', '3', ')', ',', 'Tšehhi', '6', ':', '2 , 4', ':', '6 , 7', ':', '6', ',', 'Martina', 'Hingis', '(', '1', ')', ',', 'Šveits-Arantxa', 'Sanchez', 'Vicario', '(', '10', ')', ',', 'Hispaania', '6', ':', '3 , 6', ':', '2 .'], [{'type': 'sentence_with_comma_separated_list'}]),
EnvelopingSpan(['Gigi', 'Fernandez', ',', 'USA', '/', 'Nataša', 'Zvereva', ',', 'Valgevene-', 'Alexandra', 'Fusai', '/', 'Nathalie', 'Tauziat', ',', 'Prantsusmaa', '4', ':', '6 , 6', ':', '2 , 6', ':', '2', ',', 'Nicole', 'Arendt', ',', 'USA', '/', 'Manon', 'Bollegraf', ',', 'Holland-Ruxandra', 'Dragomir', ',', 'Rumeenia', '/', 'Iva', 'Majoli', ',', 'Horvaatia', '6', ':', '3 , 3', ':', '6 , 6', ':', '4 .'], [{'type': 'sentence_with_comma_separated_list'}])])

#### Flag  `ignore_sentences_consisting_of_numbers`

Detects sentences that only contain number or numbers, no letters, and do not end with '!' nor '?'.

### Patterns for ignoring groups of consecutive sentences

#### Flag  `ignore_consecutive_parenthesized_sentences `

If consecutive sentences all contain parenthesized content that is already ignored, and all of these sentences contain less than 3 consecutive lowercase words, then these sentences likely represent enumerations (e.g. sports results) which can be safely ignored.

In [10]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''Eile õhtul sõidetud avakatsel sai Markko Märtin ( Subaru , pildil ) viienda aja .
Tulemused :
1. Tommi Mäkinen ( FIN ) Mitsubishi - 3.46 , 9
2. Marcus Grönholm ( FIN ) Peugeot +1,0
3. Harri Rovanperä ( FIN ) Peugeot +4,1
4. Carlos Sainz ( ESP ) Ford +6,0''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_consecutive_parenthesized_sentences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['Tulemused', ':', '1.'], [{'type': 'consecutive_enum_ucase_sentences'}]),
EnvelopingSpan(['Tommi', 'Mäkinen', '(', 'FIN', ')', 'Mitsubishi', '-', '3.46 , 9', '2.'], [{'type': 'consecutive_parenthesized_sentences'}]),
EnvelopingSpan(['Marcus', 'Grönholm', '(', 'FIN', ')', 'Peugeot', '+1,0', '3.'], [{'type': 'consecutive_parenthesized_sentences'}]),
EnvelopingSpan(['Harri', 'Rovanperä', '(', 'FIN', ')', 'Peugeot', '+4,1', '4.'], [{'type': 'consecutive_parenthesized_sentences'}]),
EnvelopingSpan(['Carlos', 'Sainz', '(', 'ESP', ')', 'Ford', '+6,0'], [{'type': 'consecutive_parenthesized_sentences'}])])

#### Flag  `ignore_consecutive_enum_ucase_num_sentences`

Detects sentences that: 1) start with an uppercase letter, or an ordinal number followed by an uppercase letter, or an ordinal number; 2) contain at least one number; 3) does not contain 3 consecutive lowercase words; 4) are a part of at least 4 consecutive sentences that have the same properties (1, 2 and 3).

In [11]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''Maakonniti summeerides on tulumaksu laekumise viis esimest :
1. Harjumaa 2792 ,
2. Hiiumaa 2119 ,
3. Tartumaa 2081 ,
4. Läänemaa 1933 ,
5. Pärnumaa 1903.''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_consecutive_enum_ucase_num_sentences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['Harjumaa', '2792', ',', '2.'], [{'type': 'consecutive_enum_ucase_sentences'}]),
EnvelopingSpan(['Hiiumaa', '2119', ',', '3.'], [{'type': 'consecutive_enum_ucase_sentences'}]),
EnvelopingSpan(['Tartumaa', '2081', ',', '4.'], [{'type': 'consecutive_enum_ucase_sentences'}]),
EnvelopingSpan(['Läänemaa', '1933', ',', '5.'], [{'type': 'consecutive_enum_ucase_sentences'}]),
EnvelopingSpan(['Pärnumaa', '1903.'], [{'type': 'consecutive_enum_ucase_sentences'}])])

## SyntaxIgnoreCutter

SyntaxIgnoreCutter cuts the input Text object into a smaller Text by leaving out all spans from the syntax_ignore layer. 

Usage example:

In [12]:
from estnltk.taggers.standard.syntax.preprocessing.syntax_ignore_tagger import SyntaxIgnoreTagger
from estnltk.taggers.standard.syntax.preprocessing.syntax_ignore_cutter import SyntaxIgnoreCutter
syntax_ignore_tagger = SyntaxIgnoreTagger()
syntax_ignore_cutter = SyntaxIgnoreCutter()

In [13]:
# Create input text
from estnltk import Text
text = Text('Need seminarid on toimunud Tartus 6 korda (1997–2000, 2002, 2005). '+\
            'Tutvustamisele tulevad Jan Kausi romaan “Koju” (Tuum 2012) ning Ülo Pikkovi romaan “Vana prints” (Varrak 2012).')
text.tag_layer('sentences')
# Detect ignorable parts
syntax_ignore_tagger.tag(text)
text.syntax_ignore

Layer(name='syntax_ignore', attributes=('type',), spans=SL[EnvelopingSpan(['(', '1997', '–', '2000', ',', '2002', ',', '2005', ')'], [{'type': 'parentheses_num_range'}]),
EnvelopingSpan(['(', 'Tuum', '2012', ')'], [{'type': 'parentheses_ref_year'}]),
EnvelopingSpan(['(', 'Varrak', '2012', ')'], [{'type': 'parentheses_ref_year'}])])

In [14]:
# Remove ignorable parts
cut_text = syntax_ignore_cutter.cut(text)
cut_text

Text(text='Need seminarid on toimunud Tartus 6 korda . Tutvustamisele tulevad Jan Kausi romaan “Koju”  ning Ülo Pikkovi romaan “Vana prints” .')

Note that the cut text has a words layer, which keeps words' locations in the original text (`original_start, original_end, original_index`).
This information can be used to trace which words from the original text were analysed, and is required for carrying syntactic analysis layer over to the original text (see the next section for details).

Now, we can **apply syntactic analysis on the cut text**.
We use the default MaltParser-based analysis (which is described [here](03_syntactic_analysis_with_maltparser.ipynb)):

In [15]:
cut_text.tag_layer('maltparser_syntax')
cut_text['maltparser_syntax']

Layer(name='maltparser_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('Need', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': {'pl': '', 'n': ''}, 'head': 2, 'deprel': 'det', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('seminarid', [{'id': 2, 'lemma': 'seminar', 'upostag': 'S', 'xpostag': 'S', 'feats': {'pl': '', 'n': ''}, 'head': 4, 'deprel': 'nsubj', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('on', [{'id': 3, 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 4, 'deprel': 'aux', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('toimunud', [{'id': 4, 'lemma': 'toimuma', 'upostag': 'V', 'xpostag': 'V', 'feats': {'nud': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None, 'parent_span': None, 'children': <class 'tuple'>}]),
Span('Tartus', [{'id': 5, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'in': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('6', [{'id': 6, 'lemma': '6', 'upostag': 'N', 'xpostag': 'N', 'feats': {'?': ''}, 'head': 7, 'deprel': 'nummod', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('korda', [{'id': 7, 'lemma': 'kord', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'p': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('.', [{'id': 8, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 4, 'deprel': 'punct', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('Tutvustamisele', [{'id': 1, 'lemma': 'tutvustamine', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'all': ''}, 'head': 2, 'deprel': 'obl', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('tulevad', [{'id': 2, 'lemma': 'tulema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None, 'parent_span': None, 'children': <class 'tuple'>}]),
Span('Jan', [{'id': 3, 'lemma': 'Jan', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('Kausi', [{'id': 4, 'lemma': 'Kaus', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'g': ''}, 'head': 5, 'deprel': 'nmod', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('romaan', [{'id': 5, 'lemma': 'romaan', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('“', [{'id': 6, 'lemma': '“', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'deprel': 'punct', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('Koju', [{'id': 7, 'lemma': 'Koju', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 5, 'deprel': 'parataxis', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('”', [{'id': 8, 'lemma': '”', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'deprel': 'punct', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('ning', [{'id': 9, 'lemma': 'ning', 'upostag': 'J', 'xpostag': 'J', 'feats': None, 'head': 12, 'deprel': 'cc', 'deps': None, 'mis

### Carrying syntactic analysis over to the original text

Once you've analysed the cut text syntactically, you also can carry syntactic analysis over to the original text.

Example:

In [16]:
from estnltk.taggers.standard.syntax.preprocessing.syntax_ignore_cutter import add_syntax_layer_from_cut_text

# func signature hint: original_text, cut_text, syntax_layer_name, add_empty_spans=True
add_syntax_layer_from_cut_text( text, cut_text, 'maltparser_syntax' )

Text(text='Need seminarid on toimunud Tartus 6 korda (1997–2000, 2002, 2005). Tutvustamisele tulevad Jan Kausi romaan “Koju” (Tuum 2012) ning Ülo Pikkovi romaan “Vana prints” (Varrak 2012).')

Note that by default, the resulting syntactic analysis layer will have annotations even for ignored words, but these annotations are filled with empty (`None`) values:

In [17]:
text['maltparser_syntax']

Layer(name='maltparser_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Need', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': {'pl': '', 'n': ''}, 'head': 2, 'deprel': 'det', 'deps': None, 'misc': None}]),
Span('seminarid', [{'id': 2, 'lemma': 'seminar', 'upostag': 'S', 'xpostag': 'S', 'feats': {'pl': '', 'n': ''}, 'head': 4, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('on', [{'id': 3, 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 4, 'deprel': 'aux', 'deps': None, 'misc': None}]),
Span('toimunud', [{'id': 4, 'lemma': 'toimuma', 'upostag': 'V', 'xpostag': 'V', 'feats': {'nud': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None}]),
Span('Tartus', [{'id': 5, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'in': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('6', [{'id': 6, 'lemma': '6', 'upostag': 'N', 'xpostag': 'N', 'feats': {'?': ''}, 'head': 7, 'deprel': 'nummod', 'deps': None, 'misc': None}]),
Span('korda', [{'id': 7, 'lemma': 'kord', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'p': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('(', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('1997', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('–', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('2000', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span(',', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('2002', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span(',', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('2005', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span(')', [{'id': None, 'lemma': None, 'upostag': None, 'xpostag': None, 'feats': None, 'head': None, 'deprel': None, 'deps': None, 'misc': None}]),
Span('.', [{'id': 8, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 4, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('Tutvustamisele', [{'id': 1, 'lemma': 'tutvustamine', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'all': ''}, 'head': 2, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('tulevad', [{'id': 2, 'lemma': 'tulema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None}]),
Span('Jan', [{'id': 3, 'lemma': 'Jan', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('Kausi', [{'id': 4, 'lemma': 'Kaus', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'g': ''}, 'head': 5, 'deprel': 'nmod', 'deps': None, 'misc': None}]),
Span('romaan', [{'id': 5, 'lemma': 'romaan', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('“', [{'id': 6, 'lemma': '“', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('Koju', [{'id': 7, 'lemma': 'Koju', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 5, 'deprel': 'parataxis', 'deps': None, 'misc': None}]),
Span('”', [{'id': 8, 'lemma': '”', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'dep

To remove ignored words altogether from the syntax layer, use the setting `add_empty_spans=False`:

In [18]:
# Remove old layer
text.pop_layer('maltparser_syntax')

# Repeat carrying over syntax_layer
add_syntax_layer_from_cut_text( text, cut_text, 'maltparser_syntax', add_empty_spans=False )

Text(text='Need seminarid on toimunud Tartus 6 korda (1997–2000, 2002, 2005). Tutvustamisele tulevad Jan Kausi romaan “Koju” (Tuum 2012) ning Ülo Pikkovi romaan “Vana prints” (Varrak 2012).')

In [19]:
# And None values are gone:
text['maltparser_syntax']

Layer(name='maltparser_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Need', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': {'pl': '', 'n': ''}, 'head': 2, 'deprel': 'det', 'deps': None, 'misc': None}]),
Span('seminarid', [{'id': 2, 'lemma': 'seminar', 'upostag': 'S', 'xpostag': 'S', 'feats': {'pl': '', 'n': ''}, 'head': 4, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('on', [{'id': 3, 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 4, 'deprel': 'aux', 'deps': None, 'misc': None}]),
Span('toimunud', [{'id': 4, 'lemma': 'toimuma', 'upostag': 'V', 'xpostag': 'V', 'feats': {'nud': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None}]),
Span('Tartus', [{'id': 5, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'in': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('6', [{'id': 6, 'lemma': '6', 'upostag': 'N', 'xpostag': 'N', 'feats': {'?': ''}, 'head': 7, 'deprel': 'nummod', 'deps': None, 'misc': None}]),
Span('korda', [{'id': 7, 'lemma': 'kord', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'p': ''}, 'head': 4, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('.', [{'id': 8, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 4, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('Tutvustamisele', [{'id': 1, 'lemma': 'tutvustamine', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'all': ''}, 'head': 2, 'deprel': 'obl', 'deps': None, 'misc': None}]),
Span('tulevad', [{'id': 2, 'lemma': 'tulema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': ''}, 'head': 0, 'deprel': 'root', 'deps': None, 'misc': None}]),
Span('Jan', [{'id': 3, 'lemma': 'Jan', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('Kausi', [{'id': 4, 'lemma': 'Kaus', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'g': ''}, 'head': 5, 'deprel': 'nmod', 'deps': None, 'misc': None}]),
Span('romaan', [{'id': 5, 'lemma': 'romaan', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'n': ''}, 'head': 2, 'deprel': 'nsubj', 'deps': None, 'misc': None}]),
Span('“', [{'id': 6, 'lemma': '“', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('Koju', [{'id': 7, 'lemma': 'Koju', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 5, 'deprel': 'parataxis', 'deps': None, 'misc': None}]),
Span('”', [{'id': 8, 'lemma': '”', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 7, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('ning', [{'id': 9, 'lemma': 'ning', 'upostag': 'J', 'xpostag': 'J', 'feats': None, 'head': 12, 'deprel': 'cc', 'deps': None, 'misc': None}]),
Span('Ülo', [{'id': 10, 'lemma': 'Ülo', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'n': ''}, 'head': 12, 'deprel': 'nmod', 'deps': None, 'misc': None}]),
Span('Pikkovi', [{'id': 11, 'lemma': 'Pikkov', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': '', 'g': ''}, 'head': 10, 'deprel': 'flat', 'deps': None, 'misc': None}]),
Span('romaan', [{'id': 12, 'lemma': 'romaan', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'n': ''}, 'head': 5, 'deprel': 'conj', 'deps': None, 'misc': None}]),
Span('“', [{'id': 13, 'lemma': '“', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 15, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('Vana', [{'id': 14, 'lemma': 'vana', 'upostag': 'A', 'xpostag': 'A', 'feats': {'sg': '', 'n': ''}, 'head': 15, 'deprel': 'amod', 'deps': None, 'misc': None}]),
Span('prints', [{'id': 15, 'lemma': 'prints', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'n': ''}, 'head': 12, 'deprel': 'parataxis', 'deps': None, 'misc': None}]),
Span('”', [{'id': 16, 'lemma': '”', 'upostag': 'Z', 'xpostag': 'Z', 'feats': None, 'head': 15, 'deprel': 'punct', 'deps': None, 'misc': None}]),
Span('.', [{'id': 17, 'lemma':